In [ ]:
!pip3 install --no-cache-dir tensorflow-rocm

In [1]:
!pip3 install tensorflow

In [ ]:
!pip3 install numpy
!pip3 install nltk

In [2]:
import math
import numpy as np
import string
import json
import os
import sys
import logging
import nltk
import random
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [ ]:
print(device_lib.list_local_devices())

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/vicotrbb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vicotrbb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
chars = ''
maxlen = 60
step = 3

In [64]:
print(tf.keras.__version__)
print(tf.__version__)

print(np.__version__)
print(nltk.__version__)

2.4.0
2.3.1
1.18.5
3.5


In [6]:
# Fix seed's
os.environ['PYTHONHASHSEED']=str(66)
tf.random.set_seed(66)
np.random.seed(66)
random.seed(66)

In [61]:
from collections import Mapping, Container
from sys import getsizeof
 
def deep_getsizeof(o, ids):
    d = deep_getsizeof
    if id(o) in ids:
        return 0
 
    r = getsizeof(o)
    ids.add(id(o))
 
    if isinstance(o, str):
        return r / 1048576
 
    if isinstance(o, Mapping):
        return (r + sum(d(k, ids) + d(v, ids) for k, v in o.iteritems())) / 1048576
 
    if isinstance(o, Container):
        return (r + sum(d(x, ids) for x in o)) / 1048576
    
    return r / 1048576

In [26]:
def prepareData(dataFile, parts):
    f = open(dataFile,)
    data = json.load(f)

    content = list(data[x] for x in data.keys())
    text = ''

    for c in content[:parts]:
        for i in c:
            text += i

    print(f'Corpus length: {len(text)}')

    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    text = ''
    for c in words:
        text += c
        text += ' '
    text = text.strip()

    print(f'Finished to load file')
    return text

In [27]:
text = prepareData('projects/wikipedia-nlp/wikipedia-content-dataset.json', 350)

Corpus length: 1359108
Finished to load file


In [62]:
print("Memory size: %.2f MB" % deep_getsizeof(text, set()))

Memory size: 1.80 MB


In [49]:
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Numero de sequencias:', len(sentences))

Numero de sequencias: 314412


In [50]:
chars = sorted(list(set(text)))
print('Caracteres unicos:', len(chars))

Caracteres unicos: 277


In [51]:
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vetorizando o texto')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Finalizado vetorização do texto')

print("X shape: ", x.shape)
print("Y shape: ", y.shape)

Vetorizando o texto
Finalizado vetorização do texto
X shape:  (314412, 60, 277)
Y shape:  (314412, 277)


In [63]:
print("Memory size of x: %.2f MB" % deep_getsizeof(x, set()))
print("Memory size of y: %.2f MB" % deep_getsizeof(y, set()))
print('\n')
print("Memory size of x: %.2f GB" % (deep_getsizeof(x, set())  / 1024))
print("Memory size of y: %.2f GB" % (deep_getsizeof(y, set()) / 1024))

Memory size of x: 4983.45 MB
Memory size of y: 83.06 MB


Memory size of x: 4.87 GB
Memory size of y: 0.08 GB


In [ ]:
model = Sequential([
    LSTM(len(chars), return_sequences=True, input_shape=(maxlen, len(chars))),
    LSTM(len(chars), return_sequences=True),
    LSTM(len(chars)),
    Dense(len(chars), activation='relu'),
    Dense(len(chars), activation='softmax')
])
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print("Treino do modelo")
model.fit(x, y, batch_size=128, epochs=75)